In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error


In [8]:
import pandas as pd

train = pd.read_csv("/kaggle/input/house-prices/train.csv")
test = pd.read_csv("/kaggle/input/house-prices/test.csv")

print(train.shape)
print(test.shape)


(1460, 81)
(1459, 80)


In [14]:
# Numeric columns (excluding target)
num_cols = train.select_dtypes(include=['int64', 'float64']).columns.drop("SalePrice")

# Fill numeric missing values
train[num_cols] = train[num_cols].fillna(train[num_cols].median())
test[num_cols] = test[num_cols].fillna(test[num_cols].median())

# Categorical columns
cat_cols = train.select_dtypes(include=['object']).columns
for col in cat_cols:
    train[col] = train[col].fillna(train[col].mode()[0])
    test[col] = test[col].fillna(test[col].mode()[0])

# Label Encoding
from sklearn.preprocessing import LabelEncoder
for col in cat_cols:
    lbl = LabelEncoder()
    train[col] = lbl.fit_transform(train[col].astype(str))
    test[col] = lbl.transform(test[col].astype(str))


In [16]:
X = train.drop("SalePrice", axis=1)
y = train["SalePrice"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_val)
print("Linear Regression RMSE:", np.sqrt(mean_squared_error(y_val, y_pred_lr)))


Linear Regression RMSE: 35344.207068648015


In [18]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_val)
print("Random Forest RMSE:", np.sqrt(mean_squared_error(y_val, y_pred_rf)))


Random Forest RMSE: 29001.533674561608


In [19]:
xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_val)
print("XGBoost RMSE:", np.sqrt(mean_squared_error(y_val, y_pred_xgb)))


XGBoost RMSE: 25732.21890888963


In [20]:
scores = cross_val_score(rf, X, y, cv=5, scoring='neg_root_mean_squared_error')
print("Cross-Validation RMSE:", -scores.mean())


Cross-Validation RMSE: 29674.275430996648


In [22]:
final_preds = xgb.predict(test)
submission = pd.DataFrame({
    "Id": test["Id"],
    "SalePrice": final_preds
})
submission.to_csv("submission.csv", index=False)
